# Term Project - Statistics for Data Science
## Exploratory Data Analysis
### Group 6
* Kalyan Bondada
* Shitiz Kumar Goyal
* Carlos Hinrichsen Picand
* Priyanka Kamboj
* Katya Ramos


# Summary

This notebook has the following structure:
1. __General Information of Data Set__
2. __Exploratory Analysis__
3. __Classification Analysis__

# I. General Information of Data Set

### Default of Credit Card Clients Data Set

In our term project we will use the data set of default credit card clients, provided in the webpage of the UCI Machine Learning Repository: https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients

### Data Set Information:

1. The default variable, `default payment next month`, is the response variable (Y)

On the other hand, we have the following potential predictors:

1. `LIMIT_BAL`: Amount of the given credit (NT dollar): it includes both the individual consumer credit and his/her family (supplementary) credit
2. `SEX`: (1 = male; 2 = female)
3. `EDUCATION`: (1 = graduate school; 2 = university; 3 = high school; 4 = others) 
4. `MARRIAGE`: (1 = married; 2 = single; 3 = others) 
5. `AGE`: (year). 
6. History of past payment. We tracked the past monthly payment records (from April to September, 2005) as follows: `PAY_0` = the repayment status in September, 2005; `PAY_2` = the repayment status in August, 2005; . . .;`PAY_6` = the repayment status in April, 2005. The measurement scale for the repayment status is: -1 = pay duly; 1 = payment delay for one month; 2 = payment delay for two months; . . .; 8 = payment delay for eight months; 9 = payment delay for nine months and above. 
7. Amount of bill statement (NT dollar). `BILL_AMT1` = amount of bill statement in September, 2005; `BILL_AMT2` = amount of bill statement in August, 2005; . . .; `BILL_AMT6` = amount of bill statement in April, 2005. 
8. Amount of previous payment (NT dollar). `PAY_AMT1` = amount paid in September, 2005; `PAY_AMT2` = amount paid in August, 2005; . . .;`PAY_AMT6` = amount paid in April, 2005. 

__Below you will find a series of steps in order to understand the data:__

# II. Exploratory Analysis

## 1. Import Required Libraries

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math
import random
import pandas as pd
plt.style.use('ggplot')

## 2. Upload the `default of credit card clients.xls` data to explore

In [2]:
import pandas as pd
odata = pd.read_excel('../data/raw/default of credit card clients.xls', header=[1],sheet_name="Data")

FileNotFoundError: [Errno 2] No such file or directory: 'default of credit card clients.xls'

## 3. Exploring the raw data set

The names of the different variables is shown below.

In [ ]:
# Extract column names
odata.columns

Now, we will review the general characteristics of the dataset, and then review the data.

In [ ]:
# Extract information of dataset
odata.info()

Looking for the general description of the data

In [ ]:
# Extract description
odata.describe()

Important things to notice:

__1. There are no NAN observations in the dataset__ 

__2. All the variables are saved as int. Clearly most of the variables are categorical (not considering cases like `LIMIT_BAL` and bill and payment amounts)__

__3. The ID variable is not relevant. It should be removed__

__4. We will change the name of the variable `default payment next month` to `DEFAULT`__

__5. In the variables `EDUCATION`, `MARRIAGE`, and the different categories of Payments, have more categories as the described in the dataset description webpage. We will need to review each variable in order to understand what is happening__

__The most important statistic that we can see is that only__ __$22.12$%__ __of the people are in the default category.__

In [ ]:
# Showing the data
odata

## 4. Data Cleaning

As explained above, we will first drop the irrelevant variables: ID, and rename the Default variable:

In [ ]:
# Dropping the columns
odata = odata.drop(['ID'], axis=1)

In [ ]:
# Remaning the default variable name
odata.rename(columns={'default payment next month':'DEFAULT'},inplace=True)

In [ ]:
odata.info()

Now we will review the variables that have more categories as explained in the data set information

First, `EDUCATION`:

In [ ]:
odata.EDUCATION.value_counts()/odata.EDUCATION.size*100

We can see that the categories described in the dataset information, 1 to 4, corresponds to over $95\%$ of the data. Therefore we will remove the other categories 

In [ ]:
odata = odata[(odata.EDUCATION!=0) & (odata.EDUCATION!=5) & (odata.EDUCATION!=6)]
odata.EDUCATION.value_counts()

Second, `MARRIAGE`:

In [ ]:
odata.MARRIAGE.value_counts()/odata.MARRIAGE.size*100

We can see that the categories described in the dataset information, 1 to 3, corresponds to over $95\%$ of the data. Therefore we will remove the othe category 

In [ ]:
odata = odata[(odata.MARRIAGE!=0)]
odata.MARRIAGE.value_counts()

Third, payment categories:
1. `PAY_0`

In [ ]:
odata.PAY_0.value_counts()/odata.PAY_0.size*100

We can see that the categories __NOT__ described in the dataset information, -2 and 0, corresponds to over $55\%$ of the data. Therefore we can't remove these categories. We will need further analysis. 

Before this, we will review the other payments categories to see if this behaviour is the same in the other variables

2. `PAY_2`

In [ ]:
odata.PAY_2.value_counts()/odata.PAY_2.size*100

3. `PAY_3`

In [ ]:
odata.PAY_3.value_counts()/odata.PAY_3.size*100

4. `PAY_4`

In [ ]:
odata.PAY_4.value_counts()/odata.PAY_4.size*100

5. `PAY_5`

In [ ]:
odata.PAY_5.value_counts()/odata.PAY_5.size*100

6. `PAY_6`

In [ ]:
odata.PAY_6.value_counts()/odata.PAY_6.size*100

We can see that the effect of these two categories is getting bigger in the early past payments (for example, `PAY_6`is from April 2005, and `PAY_0`is from September 2005) 

### 4.1 Payments Review

In order to understand the categories described above, we will review a subset of the data, particullarly the data related to statements and payments.

Let's subset the data:

In [ ]:
odata_s = odata.copy()
# Subset the data
odata_s = odata_s.iloc[:,5:23]
odata_s.head(15)

The first thing is to understand the logic in the payment information: As credit card users, we know that regullarly we receive our statement in one month, let's say $May 15^{th}$, and the due dat is usually one month before, in this case $June 15^{th}$. So usually there is a lag in time about our payment information, and our dataset should reflect that. For this purpose, let's look the individual #11 of the table above:  

* The variable `BILL_AMT2` is the bill amount of August 2005, which is $21670$. Following the logic, we should pay this bill statement on September 2005, which is the variable `PAY_AMT1` and corresponds to $21818$. In this case the bill was full payed, so the variable `PAY_2` should reflect that. In fact is true because the uput of the variable is -1, therefore the bill way pay in time. If the see the other statements and payments, the logic remains.

Let's see this effect in a plot:


In [ ]:
plt.rc('figure', figsize=(8, 8))
pl_pay0 = sns.scatterplot(x="BILL_AMT2", y="PAY_AMT1",hue="PAY_2", size="PAY_2",sizes=(20, 300), palette="Set2",legend="full", alpha=0.5,data=odata_s)
plt.plot(odata_s.BILL_AMT2, odata_s.BILL_AMT2, '--r')

We can see that most of the cases are plotted correctly according to tha variable `PAY_1`. Additionally, there are cases where the `BILL_AMT2` is negative, therefore it's a credit or it's an error. This same observations is shown in the following plots. 

Now let's be suspicuious about this, and think that the variables are defined one by one, I mean that the variables `BILL_AMT1`is related to `PAY_AMT1`and `PAY_0`. The plot is the following:



In [ ]:
plt.rc('figure', figsize=(8, 8))
pl_pay0 = sns.scatterplot(x="BILL_AMT1", y="PAY_AMT1",hue="PAY_0", size="PAY_0",sizes=(20, 300), palette="Set2",legend="full", alpha=0.5,data=odata_s)
plt.plot(odata_s.BILL_AMT1, odata_s.BILL_AMT1, '--r')

Nos we can see that the effect is worst here, because there are more observations labelled incorrectly. So, in that case we will assume that the correct logic is the one of the first plot.

Let's plot the rest of them:

Related to other `PAY` variables:

In [ ]:
plt.rc('figure', figsize=(15, 15))
plt.subplot(2,2,1)
pl_pay2 = sns.scatterplot(x="BILL_AMT3", y="PAY_AMT2",hue="PAY_3", size="PAY_3",sizes=(20, 300), palette="Set2",legend="full", alpha=0.5,data=odata_s)
plt.plot(odata_s.BILL_AMT3, odata_s.BILL_AMT3, '--r')
plt.subplot(2,2,2)
pl_pay3 = sns.scatterplot(x="BILL_AMT4", y="PAY_AMT3",hue="PAY_4", size="PAY_4",sizes=(20, 300), palette="Set2",legend="full", alpha=0.5,data=odata_s)
plt.plot(odata_s.BILL_AMT4, odata_s.BILL_AMT4, '--r')
plt.subplot(2,2,3)
pl_pay4 = sns.scatterplot(x="BILL_AMT5", y="PAY_AMT4",hue="PAY_5", size="PAY_5",sizes=(20, 300), palette="Set2",legend="full", alpha=0.5,data=odata_s)
plt.plot(odata_s.BILL_AMT5, odata_s.BILL_AMT5, '--r')
plt.subplot(2,2,4)
pl_pay5 = sns.scatterplot(x="BILL_AMT6", y="PAY_AMT5",hue="PAY_6", size="PAY_6",sizes=(20, 300), palette="Set2",legend="full", alpha=0.5,data=odata_s)
plt.plot(odata_s.BILL_AMT6, odata_s.BILL_AMT6, '--r')

As the first plot, we can see the same structure here. 

Due the lag, we can't use the variable `PAY_AMT6`.

Considering the previous analysis, we can say:

* Considering that the information of the variables `PAY` are a transformation of information of other variables, we should eliminate the variables `PAY_AMT` and `BILL_AMT`. 
* Because of the lag, it's not possible to reproduce all `PAY` variables, but these have more relevant information than `PAY_AMT` and `BILL_AMT` variables. 

Before doing that, let's plot the rest of the plots defined by the variables without the lag, just to be sure about the effects.

In [ ]:
plt.rc('figure', figsize=(15, 15))
plt.subplot(2,3,1)
sns.scatterplot(x="BILL_AMT2", y="PAY_AMT2",hue="PAY_2", size="PAY_2",sizes=(20, 300), palette="Set2",legend="full", alpha=0.5,data=odata_s)
plt.plot(odata_s.BILL_AMT2, odata_s.BILL_AMT2, '--r')
plt.subplot(2,3,2)
sns.scatterplot(x="BILL_AMT3", y="PAY_AMT3",hue="PAY_3", size="PAY_3",sizes=(20, 300), palette="Set2",legend="full", alpha=0.5,data=odata_s)
plt.plot(odata_s.BILL_AMT3, odata_s.BILL_AMT3, '--r')
plt.subplot(2,3,3)
sns.scatterplot(x="BILL_AMT4", y="PAY_AMT4",hue="PAY_4", size="PAY_4",sizes=(20, 300), palette="Set2",legend="full", alpha=0.5,data=odata_s)
plt.plot(odata_s.BILL_AMT4, odata_s.BILL_AMT4, '--r')
plt.subplot(2,3,4)
sns.scatterplot(x="BILL_AMT5", y="PAY_AMT5",hue="PAY_5", size="PAY_5",sizes=(20, 300), palette="Set2",legend="full", alpha=0.5,data=odata_s)
plt.plot(odata_s.BILL_AMT5, odata_s.BILL_AMT5, '--r')
plt.subplot(2,3,5)
sns.scatterplot(x="BILL_AMT6", y="PAY_AMT6",hue="PAY_6", size="PAY_6",sizes=(20, 300), palette="Set2",legend="full", alpha=0.5,data=odata_s)
plt.plot(odata_s.BILL_AMT6, odata_s.BILL_AMT6, '--r')

We have the same effect, therefore...

In [ ]:
# Dropping the variables
odata = odata.drop(['BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6','PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6'], axis=1)

In [ ]:
odata.head()

## 5. Data Transformation

We will do a few transformations. Mainly to create categorical variables and create another two variables which is the sum of the different `PAY` variables. The logic is the lower this number is, the better chances is not no be on default. 

Before doing any transformation, let's notice the meaning of the different `PAY` variables:

* __-2:__ No Consumption
* __-1:__ Pay duly
* __0:__ The use of revolving credit
* __N:__ Payment delay for N months (N=1,2,...,8)
* __9:__ Payment delay for 9 months and above

Therefore, more than categorical, `PAY` variables is like a scale. Basically because these numbers could reflect the risk that the bank could take. A client with no consumption means that the bank has no risk, a client with consumption buy paying duly has risk, but very low, and the same logic with the rest of the number above. Consequently, the lower the number, better for the bank considering risk.


#### 5.1. Change "Sex", "Education", "Marriage", and "Default" variables into categorical type. 

In order to check that the transformation is working, we will cretae other variables. Once we check that the trasnformation is right, we will drop the original variables 

In [ ]:
odata['SEX_C'] = odata['SEX'].apply(lambda x: 'Male' if x == 1 else 'Female')
odata['SEX_C'] = pd.Categorical(odata.SEX_C,categories=["Male","Female"])
odata['EDUCATION_C'] = odata['EDUCATION'].apply(lambda x: 'Graduate' if x == 1 else ('University' if x==2 else ('High School' if x==3 else "Others")))
odata['EDUCATION_C'] = pd.Categorical(odata.EDUCATION_C,categories=["Graduate","University","High School","Others"])
odata['MARRIAGE_C'] = odata['MARRIAGE'].apply(lambda x: 'Married' if x == 1 else ('Single' if x==2 else "Others"))
odata['MARRIAGE_C'] = pd.Categorical(odata.MARRIAGE_C,categories=["Married","Single","Others"])
odata['DEFAULT_C'] = odata['DEFAULT'].apply(lambda x: 'Default' if x == 1 else 'No Default')
odata['DEFAULT_C'] = pd.Categorical(odata.DEFAULT_C,categories=["Default","No Default"])
odata.head()

The transformation is working properly, therefore we will drop the original variables 

In [ ]:
# Dropping the variables
odata = odata.drop(['SEX','EDUCATION','MARRIAGE','DEFAULT'], axis=1)
odata.head()

The last transformation is to create categories for the Age according to statistics Canada: https://www.statcan.gc.ca/eng/concepts/definitions/age2

The idea is no be able to analize better the relation between the variables. The categories are:

1. __Youth:__ 15-24 years
2. __Adults:__ 25-64 years
3. __Senior:__ Over 65 years

In [ ]:
# Creating the categories
age_cat = pd.cut(odata.AGE, [14, 24, 64,100], include_lowest=False)
odata['AGE_C'] = age_cat
odata['AGE_C'] = pd.Categorical(odata.AGE_C)
odata.head()

Now let's drop the original variables

In [ ]:
# Dropping the variable
odata = odata.drop(['AGE'], axis=1)
odata.head()

#### 5.2. Create a new variables which is the sum of the `PAY` variables

The logic is the lower this number is, the better chances is not no be on default.

In [ ]:
# Creating new variable
odata['PAY_SUM'] = odata.PAY_0 + odata.PAY_2 + odata.PAY_3 + odata.PAY_4 + odata.PAY_5 + odata.PAY_6
odata.head()

Now we are ready to analyze the data

## 6. Data Analysis

#### 6.1. Relation between `DEFAULT_C`and `LIMIT_BAL` and `PAY_SUM`. 


First, let's count again the proportion of defaults:

In [ ]:
odata.groupby(['DEFAULT_C']).size()/len(odata.DEFAULT_C)*100

After the data cleaning, the proportion is practically the same.

Nos the plotting:

In [ ]:
plt.rc('figure', figsize=(15, 8))
plt.subplot(1,2,1)
sns.boxplot(x="DEFAULT_C", y="LIMIT_BAL",data=odata)
plt.subplot(1,2,2)
sns.boxplot(x="DEFAULT_C", y="PAY_SUM",data=odata)

Looking the plots above, it's easy to see that the new variable, `PAY_SUM` could explain better the default than the `LIMIT_BAL`

Now let's plot all the `PAY` variables:

In [ ]:
plt.rc('figure', figsize=(15, 15))
plt.subplot(2,3,1)
sns.boxplot(x="DEFAULT_C", y="PAY_0",data=odata)
plt.subplot(2,3,2)
sns.boxplot(x="DEFAULT_C", y="PAY_2",data=odata)
plt.subplot(2,3,3)
sns.boxplot(x="DEFAULT_C", y="PAY_3",data=odata)
plt.subplot(2,3,4)
sns.boxplot(x="DEFAULT_C", y="PAY_4",data=odata)
plt.subplot(2,3,5)
sns.boxplot(x="DEFAULT_C", y="PAY_5",data=odata)
plt.subplot(2,3,6)
sns.boxplot(x="DEFAULT_C", y="PAY_6",data=odata)

It's difficult to see a clear trend. But we can somewhat infer:

* In general the "No Defaults" by month are with number below 0, which is correct
* There are months when the number are represented as pay in time, but they are labelled as default. That's because in one month you can't see the whole behaviour on an individual
* The overall, `PAY_SUM`, could be enough to represent the default variable, but if needed we can use every individual element of `PAY`

#### 6.2. Relation between `DEFAULT_C` and `AGE_C`.

In [ ]:
odata.groupby(['AGE_C','DEFAULT_C']).size().unstack()/len(odata.DEFAULT_C)*100

Most of the cases occur when the persons are adults, $\approx 90$%, and the proportion of defaults it's almost the same as the whole dataset, therefore this age group could represent the dataset.

#### 6.3. Relation between `DEFAULT_C` and `SEX_C`.

In [ ]:
odata.groupby(['SEX_C','DEFAULT_C']).size().unstack()/len(odata.DEFAULT_C)*100

We can see that there are more females than males ($60$% vs $40$%). Additionally, the males have a default tendency higher than females. Therefore, this variable could be good for the logistic regression. 

#### 6.4. Relation between `DEFAULT_C` and `EDUCATION_C`.

In [ ]:
odata.groupby(['EDUCATION_C','DEFAULT_C']).size().unstack()/len(odata.DEFAULT_C)*100

It's not uncommon to think that the highest proportion of Default comes with lower education. Therefore, this also could be a good explanatory variable

#### 6.5. Relation between `DEFAULT_C` and `MARRIAGE_C`.

In [ ]:
odata.groupby(['MARRIAGE_C','DEFAULT_C']).size().unstack()/len(odata.DEFAULT_C)*100

Highest proportion comes to single persons.

#### 6.6. Relation between `DEFAULT_C` with `SEX_C` and `AGE_C`.

In [ ]:
odata.groupby(['SEX_C','AGE_C','DEFAULT_C']).size().unstack()/len(odata.DEFAULT_C)*100

Here we can also see that the proportion of default of adult males is bigger than females. 

Additionally, youth females have higher proportion than youth males.

#### 6.7. Relation between `DEFAULT_C` with `EDUCATION_C` and `MARRIAGE_C`.

In [ ]:
odata.groupby(['EDUCATION_C','MARRIAGE_C','DEFAULT_C']).size().unstack()/len(odata.DEFAULT_C)*100

Most of the married couples hold a University degree, and the single persons in this group are almost the same quantity ($\approx 23$%). University singles have more tendency to default than graduate single ones.

In the married couples, the the University ones have more tendency to default than graduate ones.

#### 6.8. Relation between `LIMIT_BAL` and `PAY_SUM`.

In [ ]:
plt.rc('figure', figsize=(7, 7))
sns.scatterplot(x="PAY_SUM",y="LIMIT_BAL", data=odata)

It's interesting to see that highest `LIMIT_BAL` not always have lowest `PAY_SUM`, which means a huge risk for the bank (look the point on the far right with a `PAY_SUM` over 30 and `LIMIT_BAL` over 500000). 

We can also see that there are scenarios with lowest `LIMIT_BAL` and very low default scenarios (low `PAY_SUM`). In this scenarios there is a potential opportunity to offer to the clients to upgrade the `LIMIT_BAL` according to their good payments behavior.

We can see the relation between `LIMIT_BAL `with every `PAY` variable using a correlation matrix.

In [ ]:
# Copy dataset 
odata_p =odata.copy()
# Subset according
odata_p = odata_p.loc[:, ["LIMIT_BAL","PAY_0","PAY_2","PAY_3","PAY_4","PAY_5","PAY_6","PAY_SUM"]]
plt.figure(figsize=(12, 8))
sns.heatmap(odata_p.corr().round(4), annot=True,linewidths=1.0,vmin=-1,vmax=1,cmap="coolwarm")

It's interesting to see that each individual `PAY` category has a higher correlation with its adyacent neighbor. In other words, the highest correlation of `PAY_0` is with `PAY_1`, `PAY_2` is with `PAY_3`, etc. So, your payment behaviour has a trend between the months, if you pay on time one month it's likely that you will pay on time the next month. The same case when you are not paying on time.

Another thing that we have to notice is that `PAY_SUM` has a higher correlation to `LIMIT_BAL` than any other individual payment categories.

#### 6.9. Relation between `LIMIT_BAL` and `PAY_SUM` with `AGE_C`.

In [ ]:
plt.rc('figure', figsize=(15, 7))
plt.subplot(1,2,1)
sns.boxplot(x="AGE_C",y="LIMIT_BAL", data=odata)
plt.subplot(1,2,2)
sns.boxplot(x="AGE_C",y="PAY_SUM", data=odata)

From the boxplot we can see that with more age is more likely to have a higher `LIMIT_BAL`. This makes sense, because usually with age people become more responsible financially.

Additionally, younger people has more chances to default, which is related to `PAY_SUM`.

#### 6.10. Relation between `LIMIT_BAL` and `PAY_SUM` with `SEX_C`.

In [ ]:
plt.rc('figure', figsize=(15, 7))
plt.subplot(1,2,1)
sns.boxplot(x="SEX_C",y="LIMIT_BAL", data=odata)
plt.subplot(1,2,2)
sns.boxplot(x="SEX_C",y="PAY_SUM", data=odata)

In this case is not possible to distingish between male and female with the `LIMIT_BAL`.

As previous analysis, males have a higher tendency to default than females.

#### 6.11. Relation between `LIMIT_BAL` and `PAY_SUM` with `EDUCATION_C`.

In [ ]:
plt.rc('figure', figsize=(15, 7))
plt.subplot(1,2,1)
sns.boxplot(x="EDUCATION_C",y="LIMIT_BAL", data=odata)
plt.subplot(1,2,2)
sns.boxplot(x="EDUCATION_C",y="PAY_SUM", data=odata)

Higher education means higher `LIMIT_BAL`.

On the other side, University and High School people have a higher tendency to default.

# III. Classification Analysis

For this section, we will use the following dataset:

In [ ]:
# Data for the analysis
data_a = odata.copy()
data_a.info()

## 1. Logistic Regression

## 2. KNN Classifier